# XGBoost

In [1]:
from datasets import get_datasets
datasets = get_datasets()

In [2]:
data = datasets.small.resplit(0.75,0,0.25)

In [23]:
import numpy as np, pandas as pd
train = data.train_split
train0 = train.X[train.y==0].copy()
train1 = train.X[train.y==1].copy()

In [24]:
train

xy(X=                var_0   var_1    var_2   var_3    var_4    var_5   var_6  \
ID_code                                                                    
train_49227   15.0292  0.0845   7.0515  9.6233  11.0724 -11.8830  5.4014   
train_101761  13.0058  5.8709   8.6504  5.9633  11.6448   5.8945  5.5764   
train_37110   16.4113  7.7975   9.7316  4.0067  10.8215  -7.0870  4.4055   
train_152074   7.6395 -4.5843   8.9218  3.6761  13.6540  -7.1728  5.6390   
train_111902  17.9596 -4.1012   8.5002  3.5160  11.4328  -7.9076  5.9981   
...               ...     ...      ...     ...      ...      ...     ...   
train_115548  10.8425 -7.6605  10.5641  8.4076  10.9474 -19.2507  4.6391   
train_42786    9.5460 -7.7474  18.6354  7.3233  13.0006  -3.2523  5.4781   
train_129969   8.3900  3.8427  14.3383  9.7263  10.5655 -12.1030  6.0262   
train_162783  17.4012  2.5168   9.6433  8.0242  12.7171  -2.6001  4.5290   
train_67210   13.3044 -2.7513  12.1481  8.8079  12.6856 -12.2704  5.4606   

      

In [39]:
# CALCULATE MEANS AND STANDARD DEVIATIONS
s = [0]*200
m = [0]*200
rmin=-5; rmax=5; 
res=501
for i in range(200):
    s[i] = np.std(train.X['var_'+str(i)])
    m[i] = np.mean(train.X['var_'+str(i)])
    

In [42]:
# CALCULATE PROB(TARGET=1 | X)
def getp(i,x):
    c = 3 #smoothing factor
    a = len( train1[ (train1['var_'+str(i)]>x-s[i]/c)&(train1['var_'+str(i)]<x+s[i]/c) ] ) 
    b = len( train0[ (train0['var_'+str(i)]>x-s[i]/c)&(train0['var_'+str(i)]<x+s[i]/c) ] )
    if a+b<500: return 0.1 #smoothing factor
    # RETURN PROBABILITY
    return a / (a+b)
    # ALTERNATIVELY RETURN ODDS
    # return a / b
    
# SMOOTH A DISCRETE FUNCTION
def smooth(x,st=1):
    for j in range(st):
        x2 = np.ones(len(x)) * 0.1
        for i in range(len(x)-2):
            x2[i+1] = 0.25*x[i]+0.5*x[i+1]+0.25*x[i+2]
        x = x2.copy()
    return x

In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# DRAW PLOTS, YES OR NO
Picture = False
# DATA HAS Z-SCORE RANGE OF -4.5 TO 4.5
rmin=-5; rmax=5; 
# CALCULATE PROBABILITIES FOR 501 BINS
res=501
# STORE PROBABILITIES IN PR
pr = 0.1 * np.ones((200,res))
pr2 = pr.copy()
xr = np.zeros((200,res))
xr2 = xr.copy()
ct2 = 0
for j in range(50):
    if Picture: plt.figure(figsize=(15,8))
    for v in range(4):
        ct = 0
        # CALCULATE PROBABILITY FUNCTION FOR VAR
        for i in np.linspace(rmin,rmax,res):
            pr[v+4*j,ct] = getp(v+4*j,m[v+4*j]+i*s[v+4*j])
            xr[v+4*j,ct] = m[v+4*j]+i*s[v+4*j]
            xr2[v+4*j,ct] = i
            ct += 1
        if Picture:
            # SMOOTH FUNCTION FOR PRETTIER DISPLAY
            # BUT USE UNSMOOTHED FUNCTION FOR PREDICTION
            pr2[v+4*j,:] = smooth(pr[v+4*j,:],res//10)
            # DISPLAY PROBABILITY FUNCTION
            plt.subplot(2, 4, ct2%4+5)
            plt.plot(xr[v+4*j,:],pr2[v+4*j,:],'-')
            plt.title('P( t=1 | var_'+str(v+4*j)+' )')
            xx = plt.xlim()
            # DISPLAY TARGET DENSITIES
            plt.subplot(2, 4, ct2%4+1)            
            sns.distplot(train0['var_'+str(v+4*j)], label = 't=0')
            sns.distplot(train1['var_'+str(v+4*j)], label = 't=1')
            plt.title('var_'+str(v+4*j))
            plt.legend()
            plt.xlim(xx)
            plt.xlabel('')
        if (ct2%8==0): print('Showing vars',ct2,'to',ct2+7,'...')
        ct2 += 1
    if Picture: plt.show()

Showing vars 0 to 7 ...
Showing vars 8 to 15 ...
Showing vars 16 to 23 ...
Showing vars 24 to 31 ...
Showing vars 32 to 39 ...
Showing vars 40 to 47 ...
Showing vars 48 to 55 ...
Showing vars 56 to 63 ...
Showing vars 64 to 71 ...
Showing vars 72 to 79 ...
Showing vars 80 to 87 ...
Showing vars 88 to 95 ...
Showing vars 96 to 103 ...
Showing vars 104 to 111 ...
Showing vars 112 to 119 ...
Showing vars 120 to 127 ...
Showing vars 128 to 135 ...
Showing vars 136 to 143 ...
Showing vars 144 to 151 ...
Showing vars 152 to 159 ...
Showing vars 160 to 167 ...
Showing vars 168 to 175 ...
Showing vars 176 to 183 ...
Showing vars 184 to 191 ...
Showing vars 192 to 199 ...


In [44]:
def getp2(i,x):
    z = (x-m[i])/s[i]
    ss = (rmax-rmin)/(res-1)
    if res%2==0: idx = min( (res+1)//2 + z//ss, res-1)
    else: idx = min( (res+1)//2 + (z-ss/2)//ss, res-1)
    idx = max(idx,0)
    return pr[i,int(idx)]

In [45]:
from sklearn.metrics import roc_auc_score
print('Calculating 200000 predictions and displaying a few examples...')
pred = [0]*200000; ct = 0
for r in range(train.X.index.shape[0]):
    p = 0.1
    for i in range(200):
        p *= 10*getp2(i,train.X.iloc[r,i])
    if ct%25000==0: print('train',r,'has target =',train.y.iloc[r],'and prediction =',p)
    pred[ct]=p; ct += 1
print('###############')
print('Validation AUC =',roc_auc_score(train.y, pred))

Calculating 200000 predictions and displaying a few examples...
train 0 has target = 0 and prediction = 0.018506942499466907
train 25000 has target = 0 and prediction = 0.06543276319609491
train 50000 has target = 0 and prediction = 0.0016910019150256882
###############


TypeError: tuple indices must be integers or slices, not str

In [48]:
print('###############')
print('Validation AUC =',roc_auc_score(train.y[:200000], pred))

###############


ValueError: Found input variables with inconsistent numbers of samples: [52542, 200000]

In [34]:
train.X.iloc[0,1]

0.0845